In [1]:
import numpy as np

In [2]:
a = np.array([[1,3], [2,4], [3,5]])

In [3]:
b = np.array([[1,2],[3,1],[4,2],[5,3]])

In [4]:
np.sqrt(np.square(a-b[:,None,:]).sum(2))

array([[1.        , 2.23606798, 3.60555128],
       [2.82842712, 3.16227766, 4.        ],
       [3.16227766, 2.82842712, 3.16227766],
       [4.        , 3.16227766, 2.82842712]])

In [5]:
a

array([[1, 3],
       [2, 4],
       [3, 5]])

In [6]:
a_sq_sum = np.einsum('ik,ik->i',a,a)

b_sq_sum = np.einsum('ik,ik->i',b,b)

a_dot_b = a.dot(b.T)

np.sqrt(a_sq_sum[:,None]-2*a_dot_b+b_sq_sum[None,:])

array([[1.        , 2.82842712, 3.16227766, 4.        ],
       [2.23606798, 3.16227766, 2.82842712, 3.16227766],
       [3.60555128, 4.        , 3.16227766, 2.82842712]])

In [7]:
b_sq_sum.shape

(4,)

In [8]:
a

array([[1, 3],
       [2, 4],
       [3, 5]])

In [9]:
b

array([[1, 2],
       [3, 1],
       [4, 2],
       [5, 3]])

In [20]:
np.einsum('ik,jk->ijk', a,b)

array([[[ 1,  6],
        [ 3,  3],
        [ 4,  6],
        [ 5,  9]],

       [[ 2,  8],
        [ 6,  4],
        [ 8,  8],
        [10, 12]],

       [[ 3, 10],
        [ 9,  5],
        [12, 10],
        [15, 15]]])

In [21]:
import pandas as pd

In [23]:
df = pd.read_clipboard()

In [26]:
df = df.T.reset_index()

In [28]:
df.columns=['name']

In [30]:
df.name.str.split(',')

0             [Emma, F, 20799]
1           [Olivia, F, 19674]
2           [Sophia, F, 18490]
3         [Isabella, F, 16950]
4              [Ava, F, 15586]
5              [Mia, F, 13442]
6            [Emily, F, 12562]
7          [Abigail, F, 11985]
8          [Madison, F, 10247]
9        [Charlotte, F, 10048]
10           [Harper, F, 9564]
11            [Sofia, F, 9542]
12            [Avery, F, 9517]
13        [Elizabeth, F, 9492]
14           [Amelia, F, 8727]
15           [Evelyn, F, 8692]
16             [Ella, F, 8489]
17            [Chloe, F, 8469]
18         [Victoria, F, 7955]
19           [Aubrey, F, 7589]
20            [Grace, F, 7554]
21             [Zoey, F, 7358]
22          [Natalie, F, 7061]
23          [Addison, F, 6950]
24          [Lillian, F, 6869]
25         [Brooklyn, F, 6767]
26             [Lily, F, 6727]
27           [Hannah, F, 6512]
28            [Layla, F, 6428]
29         [Scarlett, F, 5965]
                 ...          
33014          [Zedrick, M, 5]
33015   

In [ ]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


In [1]:
data = open('./data/names.csv', 'r').read() 

In [ ]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [ ]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [ ]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 